In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from use_case.baseline import * 
from tests.eval import *

N = 5
payoff_i = np.random.uniform(-10, 10, (N, N))
payoff_j = np.random.uniform(-10, 10, (N, N))

# Initialize environment
N_ACTIONS = payoff_i.shape[0]
N_AGENTS = 1000
env = BaselineEnvironment(N_AGENTS, payoff_i, payoff_j, total_games = 1)

# Actual Run

In [3]:
from models.model import *
from models.trainer import *


In [4]:
# Configure the network here
parameters = ParameterSettings(
    n_agents = N_AGENTS,
    d_action = N_ACTIONS, 
    d_obs = env.obs_size, 
    d_traits = 1,
    d_beliefs = 1
)
parameters.device = "cuda" if torch.cuda.is_available() else "cpu"

model = Model(parameters)

In [5]:
equilibriua = find_pure_equilibria(payoff_i, payoff_j)

for eq in equilibriua:
    x, y = eq 
    a = (y[0] + y[1]) / 2

    print(x, a)

(1, 4) 6.872969117114271


In [6]:
evaluate_policy(model, env, 10)


    Average Return: -0.5501727431807135
    Total returns: -5.501727431807135
    
Action Distribution
[0 1 2 3 4]
[121 169 115 270 325]


In [ ]:
# Setup the training loop
training_parameters = TrainingParameters(
    actor_training_loops = 1,
    hypernet_training_loops= 50, 
    outer_loops = 200,
    actor_learning_rate= 1e-3,
    hypernet_learning_weight= 1e-3,
    experience_buffer_size = 3,
)

train_model(model, env, training_parameters)
        

Epoch 0


Hypernet Loop: 100%|██████████| 15/15 [00:02<00:00,  5.32it/s]



    Average Policy Loss 0.0010134638287127018
    Average Entropy Loss: 30.016752243041992
    Average JSD Loss: 0.0002489837061148137
    

    Average Return: -2.4484439148416923
    Total returns: -24.484439148416925
    
Action Distribution
[0 3 4]
[591 224 185]


Actor Training: 100%|██████████| 30/30 [00:04<00:00,  7.38it/s]



    Average Policy Loss: 0.007391747087240219
    Average Entropy Loss: 6.3744916915893555 
    

    Average Return: -1.7357996094592867
    Total returns: -17.357996094592867
    
Action Distribution
[0 3 4]
[458 456  86]
Epoch 1


...

Hypernet Loop: 100%|██████████| 15/15 [00:02<00:00,  6.25it/s]



    Average Policy Loss 0.0021044944878667593
    Average Entropy Loss: 29.906261444091797
    Average JSD Loss: 0.00018204063235316426
    

    Average Return: -1.6767585427657306
    Total returns: -16.767585427657306
    
Action Distribution
[0 3 4]
[509 472  19]


Actor Training: 100%|██████████| 30/30 [00:04<00:00,  7.16it/s]



    Average Policy Loss: 0.0026711933314800262
    Average Entropy Loss: 6.341925144195557 
    


KeyboardInterrupt: 